# Assignment 3: Probabilistic modeling

In this assignment, you will work with probabilistic models known as Bayesian networks to efficiently calculate the answer to probability questions concerning discrete random variables.

To help, we've provided a package called [pbnt](https://github.com/thejinxters/pbnt) that supports the representation of Bayesian networks and automatic inference (!) of marginal probabilities. Note that you need numpy to run pbnt.

This assignment is due on T-Square on October 15th by 9:35 AM.

**Warning**
-----

Due to compatibility bugs in pbnt, **this assignment requires Python 2.7 to run, which in turn requires iPython2.** So you should run this notebook using

    ipython2 notebook probability_notebook.ipynb

If you don't have iPython2 installed, you can download it [here](https://github.com/ipython/ipython/archive/rel-2.4.1.zip), unzip it, and install it using

    python setup.py install

If you don't have iPython2 installed and you don't want to have more than one version installed, you can set it up using a virtual environment (virtualenv). 

You can find instructions on how to set up a virtualenv under the following links:

- [Windows users](http://www.tylerbutler.com/2012/05/how-to-install-python-pip-and-virtualenv-on-windows-with-powershell/) (TL;DR use PowerShell)
- [Linux/Mac users](https://virtualenv.pypa.io/en/latest/installation.html) (TL;DR use pip if you can)

Once you have virtualenv installed, you should navigate to the directory containing probability_notebook.ipynb and run the command

    virtualenv .
    
which will create a subdirectory called "bin" which contains scripts to run the virtual environment. You'll then call

    source bin/activate
    
to activate the virtualenv. You'll see that your command line now looks something like this:

    (assignment_3)my-laptop:probability_assignment user_name$

From here, you can install iPython2 to your local directory by running the command

    pip2.7 install "ipython [notebook]"
    
and then you'll be able to open probability_notebook.ipynb using iPython2.

Whenever you want to quit your virtual environment, just type

    deactivate
    
and you can reactivate the environment later with the same command you used before. If you ever want to get rid of the virtualenv files entirely, just delete the "bin" folder.

In [1]:
"""Testing pbnt.
Run this before anything else
to get pbnt to work!"""
import sys
# from importlib import reload
if('pbnt/combined' not in sys.path):
    sys.path.append('pbnt/combined')
from exampleinference import inferenceExample

# Should output:
# ('The marginal probability of sprinkler=false:', 0.80102921)
#('The marginal probability of wetgrass=false | cloudy=False, rain=True:', 0.055)

inferenceExample()

('The marginal probability of sprinkler=false:', 0.80102921)
('The marginal probability of wetgrass=false | cloudy=False, rain=True:', 0.055)


Part 1: Bayesian network tutorial
-------
40 points total

To start, design a basic probabilistic model for the following system.

There's a nuclear power plant in which an alarm is supposed to ring when the core temperature, indicated by a gauge, exceeds a fixed threshold. For simplicity, we assume that the temperature is represented as either high or normal. Use the following Boolean variables in your implementation:

- A = alarm sounds
- F<sub>A</sub> = alarm is faulty
- G = gauge reading (high = True, normal = False)
- F<sub>G</sub> = gauge is faulty
- T = actual temperature (high = True, normal = False)

In addition, assume that the gauge is **more likely to fail** when the temperature is high.

You will test your implementation at the end of the section.

1a: Casting the net
--
10 points

Design a Bayesian network for this system, using pbnt to represent the nodes and conditional probability arcs connecting nodes. Don't worry about the probabilities for now. Fill out the function below to create the net.

The following command will create a BayesNode with 2 values, an id of 0 and the name "alarm":

    A_node = BayesNode(0,2,name='alarm')

You will use BayesNode.add\_parent() and BayesNode.add\_child() to connect nodes. For example, to connect the alarm and temperature nodes that you've already made (i.e. assuming that temperature affects the alarm probability):

    A_node.add_parent(T_node)
    T_node.add_child(A_node)
    
You can run probability\_tests.network\_setup\_test() to make sure your network is set up correctly.

In [2]:
from Node import BayesNode
from Graph import BayesNet
def make_power_plant_net():
    """Create a Bayes Net representation of
    the above power plant problem."""

    A_node = BayesNode(0,2,name='alarm')
    F_A_node = BayesNode(1,2,name='faulty alarm')
    G_node = BayesNode(2,2,name='gauge')
    F_G_node = BayesNode(3,2,name='faulty gauge')
    T_node = BayesNode(4,2,name='temperature')

    F_G_node.add_parent(T_node)
    T_node.add_child(F_G_node)

    G_node.add_parent(T_node)
    T_node.add_child(G_node)

    G_node.add_parent(F_G_node)
    F_G_node.add_child(G_node)

    A_node.add_parent(G_node)
    G_node.add_child(A_node)

    A_node.add_parent(F_A_node)
    F_A_node.add_child(A_node)

    nodes = [A_node,F_A_node,G_node,F_G_node,T_node]

    return BayesNet(nodes)

In [3]:
from probability_tests import network_setup_test
power_plant = make_power_plant_net()
network_setup_test(power_plant)

correct number of nodes
correct number of edges between nodes


1b: Polytrees
--
5 points

Is the network for the power plant system a polytree? Why or why not? Choose from the following answers.

In [4]:
def is_polytree():
    """Multiple choice question about polytrees."""
    choice = 'c'
    answers = {
        'a' : 'Yes, because it can be decomposed into multiple sub-trees.',
        'b' : 'Yes, because its underlying undirected graph is a tree.',
        'c' : 'No, because its underlying undirected graph is not a tree.',
        'd' : 'No, because it cannot be decomposed into multiple sub-trees.'
    }
    return answers[choice]

1c: Setting the probabilities
---
15 points

Assume that the following statements about the system are true:

1. The temperature gauge reads the correct temperature with 95% probability when it is not faulty and 20% probability when it is faulty. For simplicity, say that the gauge's "true" value corresponds with its "hot" reading and "false" with its "normal" reading, so the gauge would have a 95% chance of returning "true" when the temperature is hot and it is not faulty.
2. The alarm is faulty 15% of the time.
3. The temperature is hot (call this "true") 20% of the time.
4. When the temperature is hot, the gauge is faulty 80% of the time. Otherwise, the gauge is faulty 5% of the time.
5. The alarm responds correctly to the gauge 55% of the time when the alarm is faulty, and it responds correctly to the gauge 90% of the time when the alarm is not faulty. For instance, when it is faulty, the alarm sounds 55% of the time that the gauge is "hot" and remains silent 55% of the time that the gauge is "normal."

Knowing these facts, set the conditional probabilities for the necessary variables on the network you just built.

Using pbnt's Distribution class: if you wanted to set the distribution for P(A) to 70% true, 30% false, you would invoke the following commands.

    A_distribution = DiscreteDistribution(A_node)
    index = A_distribution.generate_index([],[])
    A_distribution[index] = [0.3,0.7]
    A_Node.set_dist(A_distribution)

If you wanted to set the distribution for P(A|G) to be

|$G$|$P(A=true| G)$|
|------|:-----:|
|T| 0.75|
|F| 0.85| 

you would invoke:

    from numpy import zeros, float32
    dist = zeros([G_node.size(), A_node.size()], dtype=float32)
    dist[0,:] = [0.15, 0.85]
    dist[1,:] = [0.25, 0.75]
    A_distribution = ConditionalDiscreteDistribution(nodes=[G_node,A_node], table=dist)
    A_node.set_dist(A_distribution)

Modeling a three-variable relationship is a bit trickier. If you wanted to set the following distribution for $P(A|G,T)$ to be

|$G$|$T$|$P(A=true| G, T)$|
|--|--|:----:|
|T|T|0.15|
|T|F|0.6|
|F|T|0.2|
|F|F|0.1|

you would invoke:

    from numpy import zeros, float32
    dist = zeros([G_node.size(), T_node.size(), A_node.size()], dtype=float32)
    dist[1,1,:] = [0.85, 0.15]
    dist[1,0,:] = [0.4, 0.6]
    dist[0,1,:] = [0.8, 0.2]
    dist[0,0,:] = [0.9, 0.1]
    A_distribution = ConditionalDiscreteDistribution(nodes=[G_node, T_node, A_node], table=dist)
    A_node.set_dist(A_distribution)

The key is to remember that 0 represents the index of the false probability, and 1 represents true.

You can check your probability distributions with probability\_tests.probability\_setup\_test().

In [5]:
from numpy import zeros, float32
import Distribution
from Distribution import DiscreteDistribution, ConditionalDiscreteDistribution
def set_probability(bayes_net):
    """Set probability distribution for each
    node in the power plant system."""

    A_node = bayes_net.get_node_by_name("alarm")
    F_A_node = bayes_net.get_node_by_name("faulty alarm")
    G_node = bayes_net.get_node_by_name("gauge")
    F_G_node = bayes_net.get_node_by_name("faulty gauge")
    T_node = bayes_net.get_node_by_name("temperature")
    nodes = [A_node, F_A_node, G_node, F_G_node, T_node]
    # TODO: set the probability distribution for each node

    #P(T)
    T_distribution = DiscreteDistribution(T_node)
    index = T_distribution.generate_index([],[])
    T_distribution[index] = [0.80,0.20]
    T_node.set_dist(T_distribution)

    #P(FA)
    F_A_distribution = DiscreteDistribution(F_A_node)
    index = F_A_distribution.generate_index([],[])
    F_A_distribution[index] = [0.85,0.15]
    F_A_node.set_dist(F_A_distribution)

    #P(FG|T)
    from numpy import zeros, float32
    dist = zeros([T_node.size(), F_G_node.size()], dtype=float32)
    dist[0,:] = [0.95, 0.05]
    dist[1,:] = [0.20, 0.80]
    F_G_distribution = ConditionalDiscreteDistribution(nodes=[T_node,F_G_node], table=dist)
    F_G_node.set_dist(F_G_distribution)

    #P(G|T,FG)
    dist = zeros([T_node.size(), F_G_node.size(), G_node.size()], dtype=float32)
    dist[1,1,:] = [0.80, 0.20]
    dist[1,0,:] = [0.05, 0.95]
    dist[0,1,:] = [0.2, 0.8]#
    dist[0,0,:] = [0.95, 0.05]#
    G_distribution = ConditionalDiscreteDistribution(nodes=[T_node, F_G_node, G_node], table=dist)
    G_node.set_dist(G_distribution)

    #P(A|G,FA)
    dist = zeros([G_node.size(), F_A_node.size(), A_node.size()], dtype=float32)
    dist[1,1,:] = [0.45, 0.55]
    dist[1,0,:] = [0.10, 0.90]
    dist[0,1,:] = [0.55, 0.45]#
    dist[0,0,:] = [0.9, 0.1]#
    A_distribution = ConditionalDiscreteDistribution(nodes=[G_node, F_A_node, A_node], table=dist)
    A_node.set_dist(A_distribution)

    return bayes_net

In [6]:
set_probability(power_plant)
from probability_tests import probability_setup_test
probability_setup_test(power_plant)

correct temperature distribution size
correct temperature distribution
correct faulty gauge distribution size
correct faulty gauge distribution
correct gauge distribution size
correct alarm distribution size
correct faulty alarm distribution size
correct faulty alarm distribution


1d: Probability calculations
---
10 points

To finish up, you're going to perform inference on the network to calculate the following probabilities:

- the marginal probability that the alarm sounds
- the marginal probability that the gauge shows "hot"
- the probability that the temperature is actually hot, given that the alarm sounds and the alarm and gauge are both working

You'll fill out the "get_prob" functions to calculate the probabilities.

Here's an example of how to do inference for the marginal probability of the "faulty alarm" node being True (assuming "bayes_net" is your network):

    F_A_node = bayes_net.get_node_by_name('faulty alarm')
    engine = JunctionTreeEngine(bayes_net)
    Q = engine.marginal(F_A_node)[0]
    index = Q.generate_index([True],range(Q.nDims))
    prob = Q[index]
  
To compute the conditional probability, set the evidence variables before computing the marginal as seen below (here we're computing $P(A = false | F_A = true, T = False)$):

    engine.evidence[F_A_node] = True
    engine.evidence[T_node] = False
    Q = engine.marginal(A_node)[0]
    index = Q.generate_index([False],range(Q.nDims))
    prob = Q[index]

If you need to sanity-check to make sure you're doing inference correctly, you can run inference on one of the probabilities that we gave you in 1c. For instance, running inference on $P(T=true)$ should return 0.19999994 (i.e. almost 20%). You can also calculate the answers by hand to double-check.

In [7]:
def get_alarm_prob(bayes_net, alarm_rings):
    """Calculate the marginal
    probability of the alarm
    ringing (T/F) in the
    power plant system."""
    A_node = bayes_net.get_node_by_name('alarm')
    engine = JunctionTreeEngine(bayes_net)
    Q = engine.marginal(A_node)[0]
    index = Q.generate_index([alarm_rings],range(Q.nDims))
    alarm_prob = Q[index]
    return alarm_prob

In [8]:
def get_gauge_prob(bayes_net, gauge_hot):
    """Calculate the marginal
    probability of the gauge
    showing hot (T/F) in the
    power plant system."""
    G_node = bayes_net.get_node_by_name('gauge')
    engine = JunctionTreeEngine(bayes_net)
    Q = engine.marginal(G_node)[0]
    index = Q.generate_index([gauge_hot],range(Q.nDims))
    gauge_prob = Q[index]
    return gauge_prob

In [9]:
from Inference import JunctionTreeEngine
def get_temperature_prob(bayes_net,temp_hot):
    """Calculate theprobability of the
    temperature being hot (T/F) in the
    power plant system, given that the
    alarm sounds and neither the gauge
    nor alarm is faulty."""
    T_node = bayes_net.get_node_by_name('temperature')
    A_node = bayes_net.get_node_by_name('alarm')
    F_A_node = bayes_net.get_node_by_name('faulty alarm')
    F_G_node = bayes_net.get_node_by_name('faulty gauge')
    engine = JunctionTreeEngine(bayes_net)
    engine.evidence[A_node] = True
    engine.evidence[F_A_node] = False
    engine.evidence[F_G_node] = False
    Q = engine.marginal(T_node)[0]
    index = Q.generate_index([temp_hot],range(Q.nDims))
    temp_prob = Q[index]
    return temp_prob

In [10]:
get_alarm_prob(power_plant,True)
get_gauge_prob(power_plant,True)
get_temperature_prob(power_plant,True)

0.24431819

Part 2: Sampling
-----
60 points total

For the main exercise, consider the following scenario.

There are three frisbee teams who play each other: the Airheads, the Buffoons, and the Clods (A, B and C for short). Each match is between two teams, and each team can either win, lose, or draw in a match. Each team has a fixed but unknown skill level, represented as an integer from 0 to 3. Each match's outcome is probabilistically proportional to the difference in skill level between the teams.

We want to predict the outcome of the matches, given prior knowledge of previous matches. Rather than using inference, we will do so by sampling the network using two [Markov Chain Monte Carlo](http://www.statistics.com/papers/LESSON1_Notes_MCMC.pdf) models: Metropolis-Hastings (2b) and Gibbs sampling (2c).

2a: Build the network
-----
10 points

Build a Bayes Net to represent the three teams and their influences on the match outcomes. Assume the following variable conventions:

| variable name | description|
|---------|:------:|
|A| A's skill level|
|B | B's skill level|
|C | C's skill level|
|AvB | the outcome of A vs. B <br> (0 = A wins, 1 = B wins, 2 = tie)|
|BvC | the outcome of B vs. C <br> (0 = B wins, 1 = C wins, 2 = tie)|
|CvA | the outcome of C vs. A <br> (0 = C wins, 1 = A wins, 2 = tie)|

Assume that each team has the following prior distribution of skill levels:

|skill level|P(skill level)|
|----|:----:|
|0|0.15|
|1|0.45|
|2|0.30|
|3|0.10|

In addition, assume that the differences in skill levels correspond to the following probabilities of winning:

| skill difference <br> (T2 - T1) | T1 wins | T2 wins| Tie |
|------------|----------|---|:--------:|
|0|0.10|0.10|0.80|
|1|0.20|0.40|0.20|
|2|0.15|0.75|0.10|
|3|0.05|0.90|0.05|

In [11]:
def get_game_network():
    """Create a Bayes Net representation
    of the game problem."""

    A_node = BayesNode(0,4,name='A')
    B_node = BayesNode(1,4,name='B')
    C_node = BayesNode(2,4,name='C')
    AvB_node = BayesNode(3,3,name='AvB')
    BvC_node = BayesNode(4,3,name='BvC')
    CvA_node = BayesNode(5,3,name='CvA')

    AvB_node.add_parent(A_node)
    A_node.add_child(AvB_node)

    AvB_node.add_parent(B_node)
    B_node.add_child(AvB_node)

    BvC_node.add_parent(B_node)
    B_node.add_child(BvC_node)

    BvC_node.add_parent(C_node)
    C_node.add_child(BvC_node)

    CvA_node.add_parent(C_node)
    C_node.add_child(CvA_node)

    CvA_node.add_parent(A_node)
    A_node.add_child(CvA_node)


    #P(A)
    A_distribution = DiscreteDistribution(A_node)
    index = A_distribution.generate_index([],[])
    A_distribution[index] = [0.15,0.45,0.30,0.10]
    A_node.set_dist(A_distribution)

    #P(B)
    B_distribution = DiscreteDistribution(B_node)
    index = B_distribution.generate_index([],[])
    B_distribution[index] = [0.15,0.45,0.30,0.10]
    B_node.set_dist(B_distribution)

    #P(C)
    C_distribution = DiscreteDistribution(C_node)
    index = C_distribution.generate_index([],[])
    C_distribution[index] = [0.15,0.45,0.30,0.10]
    C_node.set_dist(C_distribution)

    #P(AvB|A,B)
    #P(BvC|B,C)
    #P(CvA|C,A)
    dist = zeros([A_node.size(), B_node.size(), AvB_node.size()], dtype=float32)
    dist[0,0,:] = [0.1, 0.1, 0.8]
    dist[0,1,:] = [0.2, 0.6, 0.2]
    dist[0,2,:] = [0.15, 0.75, 0.10]
    dist[0,3,:] = [0.05, 0.90, 0.05]

    dist[1,0,:] = [0.6, 0.2, 0.2]
    dist[1,1,:] = [0.1, 0.1, 0.8]
    dist[1,2,:] = [0.2, 0.6, 0.2]
    dist[1,3,:] = [0.15, 0.75, 0.10]

    dist[2,0,:] = [0.75, 0.15, 0.10]
    dist[2,1,:] = [0.6, 0.2, 0.2]
    dist[2,2,:] = [0.1, 0.1, 0.8]
    dist[2,3,:] = [0.2, 0.6, 0.2]

    dist[3,0,:] = [0.90, 0.05, 0.05]
    dist[3,1,:] = [0.75, 0.15, 0.10]
    dist[3,2,:] = [0.6, 0.2, 0.2]
    dist[3,3,:] = [0.1, 0.1, 0.8]

    AvB_distribution = ConditionalDiscreteDistribution(nodes=[A_node, B_node, AvB_node], table=dist)
    AvB_node.set_dist(AvB_distribution)

    BvC_distribution = ConditionalDiscreteDistribution(nodes=[B_node, C_node, BvC_node], table=dist)
    BvC_node.set_dist(BvC_distribution)

    CvA_distribution = ConditionalDiscreteDistribution(nodes=[C_node, A_node, CvA_node], table=dist)
    CvA_node.set_dist(CvA_distribution)

    nodes = [A_node,B_node,C_node,AvB_node,BvC_node,CvA_node]

    return BayesNet(nodes)

In [12]:
game_net = get_game_network()

2b: Metropolis-Hastings sampling
---
15 points

Now you will implement the Metropolis-Hastings algorithm, which is a method for estimating a probability distribution when it is prohibitively expensive (even for inference!) to completely compute the distribution. You'll do this in MH_sampling(), which takes a Bayesian network and initial state as a parameter and returns a sample state drawn from the network's distribution. The method should just perform a single iteration of the algorithm. If an initial value is not given, default to a state chosen uniformly at random from the possible states.

The general idea is to build an approximation of a latent probability distribution by repeatedly generating a "candidate" value for each random variable in the system, and then probabilistically accepting or rejecting the candidate value based on an underlying acceptance function. These [slides](https://www.cs.cmu.edu/~scohen/psnlp-lecture6.pdf) provide a nice intro, and this [cheat sheet](http://www.bcs.rochester.edu/people/robbie/jacobslab/cheat_sheet/MetropolisHastingsSampling.pdf) provides an explanation of the details.

Hint 1: in both Metropolis-Hastings and Gibbs sampling, you'll need access to each node's probability distribution. You can access this distribution by calling

    A_node.dist.table
    
which will return the same numpy array that you provided when constructing the probability distribution.

Hint 2: you'll also want to use the random package (e.g. random.randint()) for the probabilistic choices that sampling makes.

Hint 3: in order to count the sample states later on, you'll want to make sure the sample that you return is hashable. One way to do this is by returning the sample as a tuple.

In [13]:
def MH_sampling(bayes_net, initial_value):
    """Complete a single iteration of the
    Metropolis-Hastings algorithm given a
    Bayesian network and an initial state
    value. Returns the state sampled from
    the probability distribution."""
    init = (0,0,0,0,0,0)
    try:
        init = initial_value
    except:
        x=2 #do nothing

    A_node = bayes_net.get_node_by_name("A")
    B_node = bayes_net.get_node_by_name("B")
    C_node = bayes_net.get_node_by_name("C")
    AvB_node = bayes_net.get_node_by_name("AvB")
    BvC_node = bayes_net.get_node_by_name("BvC")
    CvA_node = bayes_net.get_node_by_name("CvA")
    A_node.value = init[0]
    B_node.value = init[1]
    C_node.value = init[2]
    AvB_node.value = init[3]
    BvC_node.value = init[4]
    CvA_node.value = init[5]

    nodes = [A_node, B_node, C_node, AvB_node, BvC_node, CvA_node]

    import random
    randprob = random.uniform(0,1)
    p = 0
    for x in range(0, A_node.numValues):
        p += A_node.dist.table[x]
        if randprob < p:
            A_node.value = x
            break
    randprob = random.uniform(0,1)
    p = 0
    for x in range(0, B_node.numValues):
        p += B_node.dist.table[x]
        if randprob < p:
            B_node.value = x
            break
    randprob = random.uniform(0,1)
    p = 0
    for x in range(0, C_node.numValues):
        p += C_node.dist.table[x]
        if randprob < p:
            C_node.value = x
            break
    randprob = random.uniform(0,1)
    p = 0
    for x in range(0, AvB_node.numValues):
        p += AvB_node.dist.table[A_node.value, B_node.value][x]
        if randprob < p:
            AvB_node.value = x
            break
    randprob = random.uniform(0,1)
    p = 0
    for x in range(0, BvC_node.numValues):
        p += BvC_node.dist.table[B_node.value, C_node.value][x]
        if randprob < p:
            BvC_node.value = x
            break
    randprob = random.uniform(0,1)
    p = 0
    for x in range(0, CvA_node.numValues):
        p += CvA_node.dist.table[C_node.value, A_node.value][x]
        if randprob < p:
            CvA_node.value = x
            break

    #print "Node value is " + str(nodeToUpdate.value)

    sample = (A_node.value, B_node.value, C_node.value, AvB_node.value, BvC_node.value, CvA_node.value)
    #print sample
    return sample

In [14]:
# arbitrary initial state for the game system
initial_value = [0,0,0,0,0,0] 
sample = MH_sampling(game_net, initial_value)

2c: Gibbs sampling
---
15 points

Implement the Gibbs sampling algorithm, which is a special case of Metropolis-Hastings. You'll do this in Gibbs_sampling(), which takes a Bayesian network and initial state value as a parameter and returns a sample state drawn from the network's distribution. The method should just consist of a single iteration of the algorithm. If no initial value is provided, default to a uniform distribution over the possible states.

You may find [this](http://gandalf.psych.umn.edu/users/schrater/schrater_lab/courses/AI2/gibbs.pdf) helpful in understanding the basics of Gibbs sampling over Bayesian networks. Make sure to identify what makes it different from Metropolis-Hastings.

In [15]:
def Gibbs_sampling(bayes_net, initial_value):
    from numpy import random
    """Complete a single iteration of the
    Gibbs sampling algorithm given a
    Bayesian network and an initial state
    value. Returns the state sampled from
    the probability distribution."""
    init = (0,0,0,0,0,0)
    try:
        init = initial_value
    except:
        x=2 #do nothing

    A_node = bayes_net.get_node_by_name("A")
    B_node = bayes_net.get_node_by_name("B")
    C_node = bayes_net.get_node_by_name("C")
    AvB_node = bayes_net.get_node_by_name("AvB")
    BvC_node = bayes_net.get_node_by_name("BvC")
    CvA_node = bayes_net.get_node_by_name("CvA")
    A_node.value = init[0]
    B_node.value = init[1]
    C_node.value = init[2]
    AvB_node.value = init[3]
    BvC_node.value = init[4]
    CvA_node.value = init[5]

    nodes = [A_node, B_node, C_node, AvB_node, BvC_node, CvA_node]

    import random
    indexToUpdate = random.randint(0,len(initial_value)-1)
    nodeToUpdate = nodes[indexToUpdate]
    
    randprob = random.uniform(0,1)
    
    if nodeToUpdate.name == 'A' or nodeToUpdate.name == 'B' or nodeToUpdate.name == 'C':
        p = 0
        for x in range(0, nodeToUpdate.numValues):
            p += nodeToUpdate.dist.table[x]
            if randprob < p:
                nodeToUpdate.value = x
                break
    else:
        nodeDistTable = []
        if nodeToUpdate.name == 'AvB':
            nodeDistTable = nodeToUpdate.dist.table[A_node.value, B_node.value]
        elif nodeToUpdate.name == 'BvC':
            nodeDistTable = nodeToUpdate.dist.table[B_node.value, C_node.value]
        else:
            nodeDistTable = nodeToUpdate.dist.table[C_node.value, A_node.value]

        p = 0
        for x in range(0, nodeToUpdate.numValues):
            p += nodeDistTable[x]
            if randprob < p:
                nodeToUpdate.value = x
                break
    #print "Node value is " + str(nodeToUpdate.value)

    sample = (A_node.value, B_node.value, C_node.value, AvB_node.value, BvC_node.value, CvA_node.value)
    #print sample
    return sample

2d: Comparing sampling methods
----
15 points

Suppose that you know the following outcome of two of the three games: A beats B and A draws with C. Start by calculating the posterior distribution for the outcome of the BvC match in calculate_posterior(), using the inference methods from 1d.

Estimate the likelihood of different outcomes for the third match by running Gibbs sampling until it converges to a stationary distribution. We'll say that the sampler has converged when, for 10 successive iterations, the difference in expected outcome for the third match differs from the previous estimated outcome by less than .1%.

Repeat this experiment for Metropolis-Hastings sampling.

Which algorithm converges more quickly? By approximately what factor? For instance, if Metropolis-Hastings takes twice as many iterations to converge as Gibbs sampling, you'd say that it converged faster by a factor of 2. Fill in sampling_question() to answer both parts.

In [16]:
from Inference import EnumerationEngine
def calculate_posterior(games_net):
    """Calculate the posterior distribution
    of the BvC match given that A won against
    B and tied C. Return a list of probabilities
    corresponding to win, loss and tie likelihood."""
    AvB_node = games_net.get_node_by_name('AvB')
    AvC_node = games_net.get_node_by_name('CvA')
    BvC_node = games_net.get_node_by_name('BvC')
    engine = EnumerationEngine(games_net)
    engine.evidence[AvB_node] = 0
    engine.evidence[AvC_node] = 2
    Q = engine.marginal(BvC_node)[0]
    return Q.table

In [17]:
iter_counts = [1e1,1e3,1e5,1e6]
def compare_sampling(bayes_net, posterior):
    """Compare Gibbs and Metropolis-Hastings
    sampling by calculating how long it takes
    for each method to converge to the
    provided posterior."""
    samplesMH = []
    initialValue = (0,0,0,0,0,0)
    counterMH = 1
    counterMHWin = 0
    counterMHLose = 0
    counterMHTie = 0
    counterMHOtherChange = 0
    convergence = []

    matchedIter = 0;
    while matchedIter < 10:
        #print "initialValue is: " + str(initialValue)
        sample = MH_sampling(bayes_net, initialValue)
        #print "Sample is: " + str(sample)
        if sample[3] == 0 and sample[5] == 2:
            currentProb = [counterMHWin/float(counterMH), counterMHLose/float(counterMH), counterMHTie/float(counterMH)]
            convergence = currentProb
            samplesMH.append(sample)
            counterMH += 1
            if sample[4] == 0:
                counterMHWin += 1
            elif sample[4] == 1:
                counterMHLose += 1
            elif sample[4] == 2:
                counterMHTie += 1

            if initialValue[4] == sample[4]:
                counterMHOtherChange += 1

            nowProb = [counterMHWin/float(counterMH), counterMHLose/float(counterMH), counterMHTie/float(counterMH)]
            difProb = [nowProb[0] - currentProb[0], nowProb[1] - currentProb[1], nowProb[2] - currentProb[2]]
            if difProb[0] < 0.0001 and difProb[1] < 0.0001 and difProb[2] < 0.0001:
                matchedIter += 1
            else:
                matchedIter = 0
        initialValue = sample
    #print "MH: " + str(counterMHWin/float(counterMH)) + " , " + str(counterMHLose/float(counterMH)) + " , " + str(counterMHTie/float(counterMH))
            #getProb(samplesMH)

    MH_convergence = [counterMHWin/float(counterMH), counterMHLose/float(counterMH), counterMHTie/float(counterMH)]


    samplesGibbs = []
    initialValue = (0,0,0,0,0,0)
    counterGibbs = 1
    counterGibbsWin = 0
    counterGibbsLose = 0
    counterGibbsTie = 0
    counterGibbsOtherChange = 0

    matchedIter = 0;
    while matchedIter < 10:
        #print "initialValue is: " + str(initialValue)
        sample = Gibbs_sampling(bayes_net, initialValue)
        #print "Sample is: " + str(sample)
        if sample[3] == 0 and sample[5] == 2: # and updatedNode == 'BvC':
            currentProb = [counterGibbsWin/float(counterGibbs), counterGibbsLose/float(counterGibbs), counterGibbsTie/float(counterGibbs)]
            samplesGibbs.append(sample)
            counterGibbs += 1
            if sample[4] == 0:
                counterGibbsWin += 1
            elif sample[4] == 1:
                counterGibbsLose += 1
            else:
                counterGibbsTie += 1

            if initialValue[4] == sample[4]:
                counterGibbsOtherChange += 1

            nowProb = [counterGibbsWin/float(counterGibbs), counterGibbsLose/float(counterGibbs), counterGibbsTie/float(counterGibbs)]
            difProb = [nowProb[0] - currentProb[0], nowProb[1] - currentProb[1], nowProb[2] - currentProb[2]]
            if difProb[0] < 0.0001 and difProb[1] < 0.0001 and difProb[2] < 0.0001:
                matchedIter += 1
            else:
                matchedIter = 0

        initialValue = sample

    Gibbs_convergence = convergence

    print "Gibbs: " + str(Gibbs_convergence)
    print "MH: " + str(MH_convergence)
    return Gibbs_convergence, MH_convergence

In [18]:
def sampling_question():
    """Question about sampling performance."""
    # TODO: assign value to choice and factor
    choice = 0
    options = ['Gibbs','Metropolis-Hastings']
    factor = 2
    return options[choice], factor

In [19]:
# test your sampling methods here
posterior = calculate_posterior(game_net)
compare_sampling(game_net, posterior)

Gibbs: [0.26537452637714953, 0.4135820460507141, 0.32089769746429614]
MH: [0.2653358589538103, 0.4136674923502841, 0.32085093982223517]


([0.26537452637714953, 0.4135820460507141, 0.32089769746429614],
 [0.2653358589538103, 0.4136674923502841, 0.32085093982223517])

2e: Theoretical follow-up
---
5 points

Suppose there are now $n$ teams in the competition, and all matches have been played except for the last match. Using inference by enumeration, how does the complexity of predicting the last match vary with $n$? 

Fill in complexity_question() to answer, using big-O notation. For example, write 'O(n^2)' for second-degree polynomial runtime.

In [20]:
def complexity_question():
    complexity = 'O(k^n)'
    return complexity

You're done! Submit this file as a "probability_notebook.ipynb" on T-Square before October 15, 9:35 AM. This assignment will be graded on the accuracy of the functions you completed.